In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct
import re
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)

In [2]:
files = !gsutil ls  gs://ds-url-catag/Stickers/sticker_bycategory/

In [ ]:
count = 0
for sub_file in files:
    sub_str = sub_file.split("/")[5]
    extract_string = sub_str[13:]   
    if extract_string != '': #ignore the first line 
        category = re.sub(r'[0-9]*','',extract_string) # get the category from the name of the file 
        try :
            path = "gs://ds-url-catag/Stickers/sticker_bycategory/"+sub_str+"/" #read in each path
            df = sc.textFile(path).map(lambda line: line.split(",")).map(lambda line: (line[0],line[1],line[2])).toDF(['id','name','animated'])
            print category 
            df = df.withColumn('category',lit(category))
            #do little bit of preprocessing
            from pyspark.sql.functions import StringType
            import re
            def removes_quotes(x):
                y = re.sub(r'["\\\\]*','',x)
                return y
            remo_udf = udf(removes_quotes, StringType())
            df = df.withColumn("sticker_pack_id",remo_udf('id'))
            df = df.withColumn("Sticker_name",remo_udf('Name'))
            #write it to a file
            df.select('sticker_pack_id','Sticker_name','category').write.mode('append').parquet("gs://ds-url-catag/Stickers/proces_sticker_bycategory/final/"+category+"/")
        except ValueError:
            count += 1

                

animalsandpets
animatedstickers
bbmfamily
characters
classics
emoji
expressions
favindonesia
food
games
loveandromance
mostpopular
movies
sports
stickerfactory
summer
tvseries
workandoffice
